In [38]:
import os
import sys
import pandas as pd
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(parent_directory)

parent /Users/kathryn/Projects/Advanced-ML/project/climate-conversations
['/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project/analysis', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/kathryn/Library/Caches/pypoetry/virtualenvs/climate-conversations-6VzO6URo-py3.11/lib/python3.11/site-packages', '/Users/kathryn/Projects/Advanced-ML/project/climate-conversations', '/Users/kathryn/Projects/Advanced-ML/project/climate-conversations']


#### All comments tokenized and classified

In [39]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_directory)
directory = os.path.join(parent_directory,'data_collection/project_data/results/comments/')
print(directory)
all_comments_tokenized_analyzed = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".pkl"):
        file_path = os.path.join(directory, filename)
        df = pd.read_pickle(file_path)

        print(f"Data from {filename}:")
        all_comments_tokenized_analyzed = pd.concat([all_comments_tokenized_analyzed, df], ignore_index=True)

print("Combined DataFrame:")
print(all_comments_tokenized_analyzed)
all_comments_tokenized_analyzed['subreddit'].unique()

/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project
/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project/data_collection/project_data/results/comments/
Data from result_climateoffensive_comment.pkl:
Data from result_climateskeptics_comment.pkl:
Data from result_climate_comment.pkl:
Data from result_climatechange_comment.pkl:
Data from result_politics_comment.pkl:
Data from result_science_comment.pkl:
Data from result_worldnews_comment.pkl:
Data from result_environment_comment.pkl:
Combined DataFrame:
              id         subreddit  \
0        iqkeli8  climateoffensive   
1        iqknoj6  climateoffensive   
2        iqknu4e  climateoffensive   
3        iqkol75  climateoffensive   
4        iqkp1b2  climateoffensive   
...          ...               ...   
1001313  iqkcv1v       environment   
1001314  iqkcz7f       environment   
1001315  iqkd2iy       environment   
1001316  iqkd43d       environment   
1001317  iqkdme3       environmen

array(['climateoffensive', 'climateskeptics', 'climate', 'climatechange',
       'politics', 'science', 'worldnews', 'environment'], dtype=object)

In [40]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_directory)
directory = os.path.join(parent_directory,'data_collection/project_data/results/submissions/')
print(directory)
all_submissions_tokenized_analyzed = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".pkl"):
        file_path = os.path.join(directory, filename)
        df = pd.read_pickle(file_path)

        print(f"Data from {filename}:")
        all_submissions_tokenized_analyzed = pd.concat([all_submissions_tokenized_analyzed, df], ignore_index=True)

print("Combined DataFrame:")
print(all_submissions_tokenized_analyzed)
all_submissions_tokenized_analyzed['subreddit'].unique()

/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project
/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project/data_collection/project_data/results/submissions/
Data from result_environment_submission.pkl:
Data from result_politics_submission.pkl:
Data from result_science_submission.pkl:
Data from result_climateskeptics_submission.pkl:
Data from result_climate_submission.pkl:
Data from result_climateoffensive_submission.pkl:
Data from result_climatechange_submission.pkl:
Data from result_worldnews_submission.pkl:
Combined DataFrame:
           id    subreddit                                              title  \
0      xsgq4s  environment  Livin Farms’ investors are betting $5.8M on po...   
1      xsirnp  environment  How Western Countries “Solutions” to Food Inse...   
2      xsjjzo  environment  Toxic algae blamed for 300 tons of dead fish i...   
3      xsndcj  environment             BP in oil field where ‘cancer is rife’   
4      xsp76t  envi

array(['environment', 'politics', 'science', 'climateskeptics', 'climate',
       'climateoffensive', 'climatechange', 'worldnews'], dtype=object)

In [41]:
final_comments = all_comments_tokenized_analyzed
final_submit = all_submissions_tokenized_analyzed



In [42]:
for full_df in (final_comments, final_submit):
    full_df['Harm_Care_Agg'] = (full_df['HarmVice'] + full_df['HarmVirtue']) / 2
    full_df['Authority_Agg'] = (full_df['AuthorityVice'] + full_df['HarmVirtue']) / 2
    full_df['Purity_Agg'] = (full_df['PurityVice'] + full_df['PurityVirtue']) / 2
    full_df['Fairness_Agg'] = (full_df['FairnessVice'] + full_df['FairnessVirtue']) / 2
    full_df['Ingroup_Agg'] = (full_df['IngroupVice'] + full_df['IngroupVirtue']) / 2
    full_df['Dominant_Moral_Foundation'] = full_df[['HarmVirtue', 'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice', 'IngroupVice', 'FairnessVirtue', 'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].idxmax(axis=1)
    full_df['Dominant_Moral_Foundation_Agg'] = full_df[['Harm_Care_Agg','Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].idxmax(axis=1)

In [43]:
final_comments.head()

,id,subreddit,body,author,score,gilded,created_utc,parent_id,link_id,retrieved_on,...,IngroupVirtue,PurityVice,PurityVirtue,Harm_Care_Agg,Authority_Agg,Purity_Agg,Fairness_Agg,Ingroup_Agg,Dominant_Moral_Foundation,Dominant_Moral_Foundation_Agg
0,iqkeli8,climateoffensive,"No, I think most people just silence their pho...",halfanothersdozen,10,0,1664582862,iqka8ek,xs6p6m,1664960539,...,0.384259,0.348129,0.331132,0.420887,0.396988,0.339630,0.345652,0.357344,HarmVirtue,Harm_Care_Agg
1,iqknoj6,climateoffensive,Google murray bookchin,skilled_cosmicist,5,0,1664587422,iqjaijq,xs6p6m,1664960259,...,0.301826,0.287511,0.288809,0.316451,0.299614,0.288160,0.155087,0.311999,HarmVirtue,Harm_Care_Agg
2,iqknu4e,climateoffensive,Don’t listen to the haters on this thread. You...,1jx,1,0,1664587502,xs6p6m,xs6p6m,1664960254,...,0.388593,0.346233,0.340704,0.417111,0.389859,0.343468,0.355956,0.359717,HarmVirtue,Harm_Care_Agg
3,iqkol75,climateoffensive,"Despite other comments, this is really smart.\...",slycrane,5,0,1664587890,xs6p6m,xs6p6m,1664960230,...,0.379215,0.344354,0.332868,0.416364,0.389709,0.338611,0.354582,0.364216,HarmVirtue,Harm_Care_Agg
4,iqkp1b2,climateoffensive,Those two examples are normalizing seatbelts a...,slycrane,4,0,1664588125,iqjc5az,xs6p6m,1664960217,...,0.384914,0.345532,0.333362,0.425554,0.392599,0.339447,0.358120,0.365197,HarmVirtue,Harm_Care_Agg


In [44]:
final_submit.columns

Index(['id', 'subreddit', 'title', 'author', 'permalink', 'num_comments',
       'url', 'score', 'gilded', 'created_utc', 'self_text', 'is_self',
       'retrieved_on', 'tokenized_title_sents', 'tokenized_title_words',
       'tokenized_title_words_norm', 'AuthorityVice', 'AuthorityVirtue',
       'FairnessVice', 'FairnessVirtue', 'HarmVice', 'HarmVirtue',
       'IngroupVice', 'IngroupVirtue', 'PurityVice', 'PurityVirtue',
       'Harm_Care_Agg', 'Authority_Agg', 'Purity_Agg', 'Fairness_Agg',
       'Ingroup_Agg', 'Dominant_Moral_Foundation',
       'Dominant_Moral_Foundation_Agg'],
      dtype='object')

In [45]:
final_comments.groupby(['Dominant_Moral_Foundation']).count()

,id,subreddit,body,author,score,gilded,created_utc,parent_id,link_id,retrieved_on,...,IngroupVice,IngroupVirtue,PurityVice,PurityVirtue,Harm_Care_Agg,Authority_Agg,Purity_Agg,Fairness_Agg,Ingroup_Agg,Dominant_Moral_Foundation_Agg
Dominant_Moral_Foundation,,,,,,,,,,,,,,,,,,,,,
AuthorityVice,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081,...,2081,2081,2081,2081,2081,2081,2081,2081,2081,2081
AuthorityVirtue,1517,1517,1517,1517,1517,1517,1517,1517,1517,1517,...,1517,1517,1517,1517,1517,1517,1517,1517,1517,1517
FairnessVice,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482,...,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482
FairnessVirtue,6456,6456,6456,6456,6456,6456,6456,6456,6456,6456,...,6456,6456,6456,6456,6456,6456,6456,6456,6456,6456
HarmVice,5208,5208,5208,5208,5208,5208,5208,5208,5208,5208,...,5208,5208,5208,5208,5208,5208,5208,5208,5208,5208
HarmVirtue,804348,804348,804348,804348,804348,804348,804348,804348,804348,804348,...,804348,804348,804348,804348,804348,804348,804348,804348,804348,804348
IngroupVice,10356,10356,10356,10356,10356,10356,10356,10356,10356,10356,...,10356,10356,10356,10356,10356,10356,10356,10356,10356,10356
IngroupVirtue,1718,1718,1718,1718,1718,1718,1718,1718,1718,1718,...,1718,1718,1718,1718,1718,1718,1718,1718,1718,1718
PurityVice,3800,3800,3800,3800,3800,3800,3800,3800,3800,3800,...,3800,3800,3800,3800,3800,3800,3800,3800,3800,3800


In [46]:
counts = final_comments.groupby(['Dominant_Moral_Foundation_Agg'])['id'].count()
print(counts)
total = counts.sum()  # Sum of all counts to get the total number of rows
percentages = (counts / total) * 100  # Divide each count by the total and multiply by 100
print(percentages)


Dominant_Moral_Foundation_Agg
Authority_Agg     12872
Fairness_Agg       7895
Harm_Care_Agg    802241
Ingroup_Agg       12908
Purity_Agg       165402
Name: id, dtype: int64
Dominant_Moral_Foundation_Agg
Authority_Agg     1.285506
Fairness_Agg      0.788461
Harm_Care_Agg    80.118504
Ingroup_Agg       1.289101
Purity_Agg       16.518429
Name: id, dtype: float64


In [47]:
subreddits = ['climate', 'climateskeptics', 'climatechange', 'environment', 'climateoffensive', 'science' ]
for sub in subreddits:
    subreddit =  final_comments[final_comments['subreddit'] == sub]
    counts = subreddit.groupby(['Dominant_Moral_Foundation_Agg'])['id'].count()
    #print(counts)
    total = counts.sum()  # Sum of all counts to get the total number of rows
    percentages = (counts / total) * 100  # Divide each count by the total and multiply by 100
    print('\n')
    print(f'Percentages for {sub}')
    print(percentages)



Percentages for climate
Dominant_Moral_Foundation_Agg
Authority_Agg     1.254530
Fairness_Agg      0.594740
Harm_Care_Agg    76.312610
Ingroup_Agg       1.198773
Purity_Agg       20.639346
Name: id, dtype: float64


Percentages for climateskeptics
Dominant_Moral_Foundation_Agg
Authority_Agg     1.258492
Fairness_Agg      0.779597
Harm_Care_Agg    95.712217
Ingroup_Agg       0.946653
Purity_Agg        1.303040
Name: id, dtype: float64


Percentages for climatechange
Dominant_Moral_Foundation_Agg
Authority_Agg     0.963534
Fairness_Agg      0.355766
Harm_Care_Agg    89.875482
Ingroup_Agg       0.756004
Purity_Agg        8.049214
Name: id, dtype: float64


Percentages for environment
Dominant_Moral_Foundation_Agg
Authority_Agg     1.727585
Fairness_Agg      0.820218
Harm_Care_Agg    93.625365
Ingroup_Agg       0.999641
Purity_Agg        2.827190
Name: id, dtype: float64


Percentages for climateoffensive
Dominant_Moral_Foundation_Agg
Authority_Agg     0.934579
Fairness_Agg      0.431344

Dominant moral foundations, comments

In [63]:
# Your list of subreddits
subreddits = ['climate', 'climateskeptics', 'climatechange', 'environment', 'climateoffensive', 'science', 'politics', 'worldnews']

# Initialize an empty DataFrame for the final result
results_df = pd.DataFrame()

# Loop through each subreddit and calculate percentages
for sub in subreddits:
    subreddit = final_comments[final_comments['subreddit'] == sub]
    counts = subreddit.groupby(['Dominant_Moral_Foundation_Agg'])['id'].count()
    total = counts.sum()  # Sum of all counts to get the total number of rows
    percentages = round((counts / total) * 100,2)  # Divide each count by the total and multiply by 100
    percentages_df = percentages.reset_index()
    percentages_df['subreddit'] = sub  # Add a column for the subreddit
    results_df = pd.concat([results_df, percentages_df], ignore_index=True)

# Rename columns for clarity
results_df.columns = ['Dominant_Moral_Foundation_Agg', 'Percentage', 'Subreddit']

# Pivot the table to get the desired format
final_table = results_df.pivot(index='Dominant_Moral_Foundation_Agg', columns='Subreddit', values='Percentage')

final_table

Subreddit,climate,climatechange,climateoffensive,climateskeptics,environment,politics,science,worldnews
Dominant_Moral_Foundation_Agg,,,,,,,,
Authority_Agg,1.25,0.96,0.93,1.26,1.73,1.81,0.65,1.39
Fairness_Agg,0.59,0.36,0.43,0.78,0.82,1.09,0.35,0.95
Harm_Care_Agg,76.31,89.88,93.75,95.71,93.63,94.96,47.50,94.91
Ingroup_Agg,1.20,0.76,0.93,0.95,1.00,1.49,0.33,2.03
Purity_Agg,20.64,8.05,3.95,1.30,2.83,0.66,51.17,0.72


Dominant Moral Foundations, submissions

In [49]:

# Your list of subreddits
subreddits = ['climate', 'climateskeptics', 'climatechange', 'environment', 'climateoffensive', 'science', 'politics']

# Initialize an empty DataFrame for the final result
results_df = pd.DataFrame()

# Loop through each subreddit and calculate percentages
for sub in subreddits:
    subreddit = final_submit[final_submit['subreddit'] == sub]
    counts = subreddit.groupby(['Dominant_Moral_Foundation_Agg'])['id'].count()
    total = counts.sum()  # Sum of all counts to get the total number of rows
    percentages = round((counts / total) * 100,2)  # Divide each count by the total and multiply by 100
    percentages_df = percentages.reset_index()
    percentages_df['subreddit'] = sub  # Add a column for the subreddit
    results_df = pd.concat([results_df, percentages_df], ignore_index=True)

# Rename columns for clarity
results_df.columns = ['Dominant_Moral_Foundation_Agg', 'Percentage', 'Subreddit']

# Pivot the table to get the desired format
final_table = results_df.pivot(index='Dominant_Moral_Foundation_Agg', columns='Subreddit', values='Percentage')

final_table

Subreddit,climate,climatechange,climateoffensive,climateskeptics,environment,politics,science
Dominant_Moral_Foundation_Agg,,,,,,,
Authority_Agg,1.39,3.07,4.62,2.82,3.35,2.61,2.45
Fairness_Agg,1.32,1.75,0.51,2.65,1.02,2.05,0.93
Harm_Care_Agg,91.09,86.57,86.15,86.07,90.97,89.50,93.31
Ingroup_Agg,5.85,6.72,7.18,7.23,3.75,5.28,2.59
Purity_Agg,0.35,1.90,1.54,1.23,0.91,0.56,0.73


In [50]:
# Your list of subreddits
subreddits = ['climate', 'climateskeptics', 'climatechange', 'environment', 'climateoffensive', 'science', 'politics', 'worldnews']

# Initialize an empty DataFrame for the final result
results_df = pd.DataFrame()

# Loop through each subreddit and calculate percentages
for sub in subreddits:
    subreddit = final_comments[final_comments['subreddit'] == sub]
    counts = subreddit.groupby(['Dominant_Moral_Foundation_Agg'])['id'].count()
    total = counts.sum()  # Sum of all counts to get the total number of rows
    percentages = round((counts / total) * 100,2)  # Divide each count by the total and multiply by 100
    percentages_df = percentages.reset_index()
    percentages_df['subreddit'] = sub  # Add a column for the subreddit
    results_df = pd.concat([results_df, percentages_df], ignore_index=True)

# Rename columns for clarity
results_df.columns = ['Dominant_Moral_Foundation_Agg', 'Percentage', 'Subreddit']

# Pivot the table to get the desired format
final_table = results_df.pivot(index='Dominant_Moral_Foundation_Agg', columns='Subreddit', values='Percentage')

final_table

Subreddit,climate,climatechange,climateoffensive,climateskeptics,environment,science
Dominant_Moral_Foundation_Agg,,,,,,
Authority_Agg,1.25,0.96,0.93,1.26,1.73,0.65
Fairness_Agg,0.59,0.36,0.43,0.78,0.82,0.35
Harm_Care_Agg,76.31,89.88,93.75,95.71,93.63,47.50
Ingroup_Agg,1.20,0.76,0.93,0.95,1.00,0.33
Purity_Agg,20.64,8.05,3.95,1.30,2.83,51.17


Submissions

In [51]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_submissions = final_submit.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_submissions)


          subreddit  HarmVirtue  AuthorityVirtue  PurityVirtue  HarmVice  \
0           climate    0.402450         0.346827      0.326239  0.350899   
1     climatechange    0.394976         0.336604      0.313901  0.338197   
2  climateoffensive    0.408775         0.348106      0.323874  0.351268   
3   climateskeptics    0.395781         0.337478      0.317850  0.340404   
4       environment    0.402189         0.344836      0.323884  0.350696   
5          politics    0.400186         0.345634      0.317917  0.349864   
6           science    0.402868         0.344177      0.318868  0.352040   
7         worldnews    0.394199         0.340860      0.316329  0.353628   

   PurityVice  IngroupVice  FairnessVirtue  FairnessVice  IngroupVirtue  \
0    0.323762     0.346630        0.366112      0.299443       0.348703   
1    0.311547     0.333248        0.355351      0.282498       0.337478   
2    0.320134     0.343219        0.367692      0.294018       0.349396   
3    0.313547  

Comments

In [52]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_comments = final_comments.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_comments)

          subreddit  HarmVirtue  AuthorityVirtue  PurityVirtue  HarmVice  \
0           climate    0.420494         0.348581      0.343346  0.356256   
1     climatechange    0.426786         0.350456      0.330264  0.358057   
2  climateoffensive    0.432727         0.352710      0.326965  0.360048   
3   climateskeptics    0.432367         0.353546      0.326054  0.361477   
4       environment    0.437061         0.353827      0.327591  0.364588   
5          politics    0.442993         0.358219      0.324821  0.368574   
6           science    0.405037         0.347026      0.377950  0.353798   
7         worldnews    0.439430         0.354844      0.322416  0.368035   

   PurityVice  IngroupVice  FairnessVirtue  FairnessVice  IngroupVirtue  \
0    0.327404     0.322053        0.363355      0.284310       0.343142   
1    0.327852     0.322713        0.371583      0.287808       0.352748   
2    0.328957     0.322967        0.375477      0.284774       0.357871   
3    0.330468  

In [53]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_comments_agg = final_comments.groupby('subreddit')[['Harm_Care_Agg',
       'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].mean().reset_index()

# Print the resulting DataFrame
print(average_comments_agg)

          subreddit  Harm_Care_Agg  Authority_Agg  Purity_Agg  Fairness_Agg  \
0           climate       0.388375       0.368395    0.335375      0.323832   
1     climatechange       0.392421       0.370328    0.329058      0.329696   
2  climateoffensive       0.396387       0.373456    0.327961      0.330126   
3   climateskeptics       0.396922       0.374216    0.328261      0.336857   
4       environment       0.400825       0.376622    0.330254      0.333350   
5          politics       0.405784       0.381344    0.328757      0.341131   
6           science       0.379417       0.365535    0.353596      0.311835   
7         worldnews       0.403733       0.378393    0.327036      0.338140   

   Ingroup_Agg  
0     0.332598  
1     0.337730  
2     0.340419  
3     0.343887  
4     0.340231  
5     0.350250  
6     0.320311  
7     0.349296  


In [54]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_submit_agg = final_submit.groupby('subreddit')[['Harm_Care_Agg',
       'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']].mean().reset_index()

# Print the resulting DataFrame
print(average_submit_agg)

          subreddit  Harm_Care_Agg  Authority_Agg  Purity_Agg  Fairness_Agg  \
0           climate       0.376675       0.358658    0.325001      0.332778   
1     climatechange       0.366587       0.346325    0.312724      0.318924   
2  climateoffensive       0.380022       0.360549    0.322004      0.330855   
3   climateskeptics       0.368092       0.348456    0.315698      0.323444   
4       environment       0.376443       0.358801    0.323875      0.329621   
5          politics       0.375025       0.356456    0.316995      0.330235   
6           science       0.377454       0.357310    0.322388      0.326924   
7         worldnews       0.373913       0.353512    0.316692      0.327545   

   Ingroup_Agg  
0     0.347667  
1     0.335363  
2     0.346308  
3     0.336377  
4     0.343460  
5     0.341880  
6     0.340190  
7     0.344001  


## Top 5 entries

In [64]:
import pandas as pd

filtered_comments = final_comments[final_comments['body'].str.len() > 15]

columns_to_sort = ['Harm_Care_Agg', 'Authority_Agg', 'Purity_Agg', 'Fairness_Agg', 'Ingroup_Agg']

top_5_entries_with_scores = {}

for column in columns_to_sort:
    top_entries = filtered_comments.sort_values(by=column, ascending=False).head(5)
    top_5_entries_with_scores[column] = top_entries[['tokenized_body_sents', column]].values.tolist()

In [56]:
top_5_entries_with_scores

{'Harm_Care_Agg': [[['we knew, we knew.'], 0.5434596836566925],
  [['gruft knows gruft, same as a bliffer knows a bliffer.'],
   0.5434596836566925],
  [['if you know you know 😬'], 0.5434596836566925],
  [['you know it', 'and i know it!'], 0.5434596836566925],
  [['do they know we know?'], 0.5434596836566925]],
 'Authority_Agg': [[['more like treason.'], 0.5051937103271484],
  [['not talking to you.', 'talking to efecto\\_vogel.'], 0.5024101585149765],
  [['he talks the talk.'], 0.5024101585149765],
  [["but he wasn't talking about zelensky, he was talking about /u/sixgun217."],
   0.5024101585149765],
  [['what are you talking about ?'], 0.5024101287126541]],
 'Purity_Agg': [[['abc: always be conning'], 0.5119925439357758],
  [['she’s a con, i don’t truss her'], 0.5119925439357758],
  [["it's the dementia"], 0.47662997245788574],
  [['https://agupubs.onlinelibrary.wiley.com/cms/asset/724a0d06-4a27-443e-8115-ec7f258ecea3/aga220157-fig-0001-m.jpg y axis'],
   0.47405828535556793],
  [['

## Filtered

In [57]:
filtered_comments = final_comments[final_comments['body'].str.len() > 20]
filtered_submit = final_submit[final_submit['title'].str.len() > 20]

In [58]:
# Group by 'subreddit' and calculate the mean for each moral foundation
average_filtered_comments = filtered_comments.groupby('subreddit')[['HarmVirtue',
       'AuthorityVirtue', 'PurityVirtue', 'HarmVice', 'PurityVice',
       'IngroupVice', 'FairnessVirtue',
       'FairnessVice', 'IngroupVirtue', 'AuthorityVice']].mean().reset_index()

# Print the resulting DataFrame
print(average_filtered_comments)

          subreddit  HarmVirtue  AuthorityVirtue  PurityVirtue  HarmVice  \
0           climate    0.440685         0.358452      0.327549  0.367550   
1     climatechange    0.436497         0.356990      0.326044  0.364862   
2  climateoffensive    0.440000         0.358628      0.326208  0.365989   
3   climateskeptics    0.435093         0.356415      0.326510  0.364135   
4       environment    0.444327         0.360125      0.328997  0.370898   
5          politics    0.448146         0.362840      0.328838  0.373406   
6           science    0.447030         0.362320      0.328398  0.372370   
7         worldnews    0.445779         0.360506      0.327419  0.374055   

   PurityVice  IngroupVice  FairnessVirtue  FairnessVice  IngroupVirtue  \
0    0.333998     0.338451        0.382851      0.300811       0.366754   
1    0.332547     0.336441        0.380527      0.301231       0.365130   
2    0.332890     0.337181        0.381280      0.299117       0.366966   
3    0.332480  

Most common word by foundation 

In [65]:
from collections import Counter, defaultdict
filtered_comments = final_comments[final_comments['body'].str.len() > 20]
# Assuming 'final_comments' is your DataFrame
# Example structure: final_comments[['Dominant_Moral_Foundations_Agg', 'tokenized_body_sent_norm']]

# Initialize a default dictionary to store concatenated lists of words for each moral foundation category
words_by_foundation = defaultdict(list)

# Aggregate words by their respective moral foundation categories
for _index, row in filtered_comments.iterrows():
    foundation = row['Dominant_Moral_Foundation_Agg']
    words = row['tokenized_body_words_norm']  # Assuming this is already a list of words
    words = [word for word in words if word not in ['~', '=', '$', '☆']]
    words_by_foundation[foundation].extend(words)

# Initialize a dictionary to store the most common words for each foundation
most_common_words_by_foundation = {}

# Calculate the most common words for each foundation
for foundation, words in words_by_foundation.items():
    word_counts = Counter(words)
    most_common_words = word_counts.most_common(10)  # Adjust the number to get more or fewer words
    most_common_words_by_foundation[foundation] = most_common_words




In [60]:
most_common_words_by_foundation

{'Harm_Care_Agg': [('people', 135297),
  ('like', 111673),
  ('think', 80017),
  ('go', 64052),
  ('know', 63533),
  ('russia', 61383),
  ('want', 59821),
  ('say', 57337),
  ('time', 57040),
  ('thing', 53269)],
 'Authority_Agg': [('ha', 1530),
  ('illegal', 554),
  ('pay', 357),
  ('traitor', 244),
  ('vote', 210),
  ('tax', 204),
  ('ban', 194),
  ('trump', 188),
  ('legal', 186),
  ('agree', 184)],
 'Fairness_Agg': [('bias', 498),
  ('member', 335),
  ('trump', 156),
  ('russia', 126),
  ('fascist', 115),
  ('leader', 110),
  ('korea', 105),
  ('gt', 97),
  ('nato', 97),
  ('source', 92)],
 'Ingroup_Agg': [('russian', 1126),
  ('⣿', 1020),
  ('⠀', 803),
  ('russia', 716),
  ('government', 546),
  ('putin', 419),
  ('military', 371),
  ('country', 350),
  ('ukraine', 337),
  ('state', 318)],
 'Purity_Agg': [('de', 125),
  ('ukraine', 117),
  ('que', 62),
  ('trump', 56),
  ('la', 53),
  ('un', 47),
  ('mlm', 46),
  ('remove', 44),
  ('o', 42),
  ('reddit', 41)]}

Most common word by subreddit

In [61]:
from collections import Counter, defaultdict
filtered_comments = final_comments[final_comments['body'].str.len() > 20]
# Assuming 'final_comments' is your DataFrame
# Example structure: final_comments[['Dominant_Moral_Foundations_Agg', 'tokenized_body_sent_norm']]

# Initialize a default dictionary to store concatenated lists of words for each moral foundation category
words_by_foundation = defaultdict(list)

# Aggregate words by their respective moral foundation categories
for _index, row in filtered_comments.iterrows():
    foundation = row['subreddit']
    words = row['tokenized_body_words_norm']  # Assuming this is already a list of words
    words = [word for word in words if word not in ['~', '=', '$', '☆']]
    words_by_foundation[foundation].extend(words)

# Initialize a dictionary to store the most common words for each foundation
most_common_words_by_subreddit = {}

# Calculate the most common words for each foundation
for foundation, words in words_by_foundation.items():
    word_counts = Counter(words)
    most_common_words = word_counts.most_common(10)  # Adjust the number to get more or fewer words
    most_common_words_by_subreddit[foundation] = most_common_words




In [62]:
most_common_words_by_foundation

{'climateoffensive': [('climate', 415),
  ('people', 368),
  ('change', 319),
  ('like', 270),
  ('need', 239),
  ('think', 234),
  ('work', 218),
  ('year', 202),
  ('thing', 199),
  ('carbon', 198)],
 'climateskeptics': [('climate', 2836),
  ('temperature', 2170),
  ('change', 2150),
  ('co2', 2033),
  ('year', 1709),
  ('energy', 1677),
  ('heat', 1628),
  ('like', 1563),
  ('surface', 1341),
  ('know', 1339)],
 'climate': [('climate', 2017),
  ('people', 1374),
  ('change', 1256),
  ('like', 1195),
  ('need', 922),
  ('year', 913),
  ('think', 804),
  ('thing', 777),
  ('fuel', 732),
  ('go', 726)],
 'climatechange': [('change', 1896),
  ('climate', 1852),
  ('co2', 1713),
  ('year', 1653),
  ('increase', 1249),
  ('energy', 976),
  ('like', 957),
  ('think', 952),
  ('people', 926),
  ('time', 864)],
 'politics': [('people', 45039),
  ('like', 35316),
  ('politic', 34923),
  ('trump', 31894),
  ('think', 25588),
  ('want', 21727),
  ('right', 21638),
  ('rule', 21161),
  ('vote', 

In [70]:
"""
Loads Processed Data for Visualization

Author(s): Kathryn Link-Oberstar
"""
import os
import pandas as pd

HURRICANE_START = pd.to_datetime('2022-09-23')
HURRICANE_END = pd.to_datetime('2022-09-30')

def load_comments():
    parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
    directory = os.path.join(
        parent_directory, "data_collection/project_data/results/comments/"
    )
    all_comments_tokenized_analyzed = directory_importer(directory)
    all_comments_tokenized_analyzed = df_processor(
        all_comments_tokenized_analyzed
    )
    return all_comments_tokenized_analyzed


def load_submissions():
    parent_directory = os.path.abspath(os.path.join(os.getcwd(), ".."))
    directory = os.path.join(
        parent_directory, "data_collection/project_data/results/submissions/"
    )
    all_submissions_tokenized_analyzed = directory_importer(directory)
    all_submissions_tokenized_analyzed = df_processor(
        all_submissions_tokenized_analyzed
    )
    return all_submissions_tokenized_analyzed


def directory_importer(directory):
    final_df = pd.DataFrame()
    for filename in os.listdir(directory):
        if filename.endswith(".pkl"):
            file_path = os.path.join(directory, filename)
            df = pd.read_pickle(file_path)
            final_df = pd.concat([final_df, df], ignore_index=True)
    return final_df


def df_processor(full_df):
    full_df["Harm_Care_Agg"] = (full_df["HarmVice"] + full_df["HarmVirtue"]) / 2
    full_df["Authority_Agg"] = (
        full_df["AuthorityVice"] + full_df["HarmVirtue"]
    ) / 2
    full_df["Purity_Agg"] = (
        full_df["PurityVice"] + full_df["PurityVirtue"]
    ) / 2
    full_df["Fairness_Agg"] = (
        full_df["FairnessVice"] + full_df["FairnessVirtue"]
    ) / 2
    full_df["Ingroup_Agg"] = (
        full_df["IngroupVice"] + full_df["IngroupVirtue"]
    ) / 2
    full_df["Dominant_Moral_Foundation"] = full_df[
        [
            "HarmVirtue",
            "AuthorityVirtue",
            "PurityVirtue",
            "HarmVice",
            "PurityVice",
            "IngroupVice",
            "FairnessVirtue",
            "FairnessVice",
            "IngroupVirtue",
            "AuthorityVice",
        ]
    ].idxmax(axis=1)
    full_df["Dominant_Moral_Foundation_Agg"] = full_df[
        [
            "Harm_Care_Agg",
            "Authority_Agg",
            "Purity_Agg",
            "Fairness_Agg",
            "Ingroup_Agg",
        ]
    ].idxmax(axis=1)

    full_df['created_utc'] = pd.to_datetime(full_df['created_utc'], unit='s')  # Convert Unix timestamp to datetime
    full_df['Hurricane_Period'] = full_df.apply(categorize_period, axis=1)
    print(full_df['Hurricane_Period'].unique())

    return full_df

def categorize_period(row):
    if row['created_utc'] < HURRICANE_START:
        return 'Before Hurricane'
    elif HURRICANE_START <= row['created_utc'] <= HURRICANE_END:
        return 'During Hurricane'
    else:
        return 'After Hurricane'

def main():
    comments_df = load_comments()
    submissions_df = load_submissions()

    print("Comments DataFrame:")
    print(comments_df.head())

    print("\nSubmissions DataFrame:")
    print(submissions_df.head())


if __name__ == "__main__":
    main()


['After Hurricane' 'Before Hurricane' 'During Hurricane']
['After Hurricane' 'Before Hurricane' 'During Hurricane']
Comments DataFrame:
        id         subreddit  \
0  iqkeli8  climateoffensive   
1  iqknoj6  climateoffensive   
2  iqknu4e  climateoffensive   
3  iqkol75  climateoffensive   
4  iqkp1b2  climateoffensive   

                                                body             author  \
0  No, I think most people just silence their pho...  halfanothersdozen   
1                             Google murray bookchin  skilled_cosmicist   
2  Don’t listen to the haters on this thread. You...                1jx   
3  Despite other comments, this is really smart.\...           slycrane   
4  Those two examples are normalizing seatbelts a...           slycrane   

   score  gilded         created_utc parent_id link_id  retrieved_on  ...  \
0     10       0 2022-10-01 00:07:42   iqka8ek  xs6p6m    1664960539  ...   
1      5       0 2022-10-01 01:23:42   iqjaijq  xs6p6m    16649602

In [68]:
full_df.columns

Index(['id', 'subreddit', 'title', 'author', 'permalink', 'num_comments',
       'url', 'score', 'gilded', 'created_utc', 'self_text', 'is_self',
       'retrieved_on', 'tokenized_title_sents', 'tokenized_title_words',
       'tokenized_title_words_norm', 'AuthorityVice', 'AuthorityVirtue',
       'FairnessVice', 'FairnessVirtue', 'HarmVice', 'HarmVirtue',
       'IngroupVice', 'IngroupVirtue', 'PurityVice', 'PurityVirtue',
       'Harm_Care_Agg', 'Authority_Agg', 'Purity_Agg', 'Fairness_Agg',
       'Ingroup_Agg', 'Dominant_Moral_Foundation',
       'Dominant_Moral_Foundation_Agg'],
      dtype='object')